In [1]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline


tokenizer = AutoTokenizer.from_pretrained("defog/sqlcoder-7b")
model = \
    AutoModelForCausalLM.from_pretrained(
        "defog/sqlcoder-7b", device_map="auto"
    )

question = "What products has the biggest fall in sales in 2022 compared to 2021? Give me the product name, the sales amount in both years, and the difference."

prompt = """### Instructions:
Your task is convert a question into a SQL query, given a Postgres database schema.
Adhere to these rules:
- **Deliberately go through the question and database schema word by word** to appropriately answer the question
- **Use Table Aliases** to prevent ambiguity. For example, `SELECT table1.col1, table2.col1 FROM table1 JOIN table2 ON table1.id = table2.id`.
- When creating a ratio, always cast the numerator as float

### Input:
Generate a SQL query that answers the questioq `{question}`.
This query will run on a database whose schema is represented in this string:
CREATE TABLE products (
  product_id INTEGER PRIMARY KEY, -- Unique ID for each product
  name VARCHAR(50), -- Name of the product
  price DECIMAL(10,2), -- Price of each unit of the product
  quantity INTEGER  -- Current quantity in stock
);

CREATE TABLE customers (
   customer_id INTEGER PRIMARY KEY, -- Unique ID for each customer
   name VARCHAR(50), -- Name of the customer
   address VARCHAR(100) -- Mailing address of the customer
);

CREATE TABLE salespeople (
  salesperson_id INTEGER PRIMARY KEY, -- Unique ID for each salesperson
  name VARCHAR(50), -- Name of the salesperson
  region VARCHAR(50) -- Geographic sales region
);

CREATE TABLE sales (
  sale_id INTEGER PRIMARY KEY, -- Unique ID for each sale
  product_id INTEGER, -- ID of product sold
  customer_id INTEGER,  -- ID of customer who made purchase
  salesperson_id INTEGER, -- ID of salesperson who made the sale
  sale_date DATE, -- Date the sale occurred
  quantity INTEGER -- Quantity of product sold
);

CREATE TABLE product_suppliers (
  supplier_id INTEGER PRIMARY KEY, -- Unique ID for each supplier
  product_id INTEGER, -- Product ID supplied
  supply_price DECIMAL(10,2) -- Unit price charged by supplier
);

-- sales.product_id can be joined with products.product_id
-- sales.customer_id can be joined with customers.customer_id
-- sales.salesperson_id can be joined with salespeople.salesperson_id
-- product_suppliers.product_id can be joined with products.product_id

### Response:
Based on your instructions, here is the SQL query I have generated to answer the question `{question}`:
```sql
""".format(question=question)

eos_token_id = tokenizer.convert_tokens_to_ids(["```"])[0]

inputs = tokenizer(prompt, return_tensors="pt").to('cuda')
generated_ids = model.generate(
    **inputs,
    num_return_sequences=1,
    eos_token_id=eos_token_id,
    pad_token_id=eos_token_id,
    max_new_tokens=400,
    do_sample=False,
    num_beams=5
)

outputs = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)
torch.cuda.empty_cache()
print(outputs[0].split("```sql")[-1].split("```")[0].split(";")[0].strip() + ";")

[2023-11-09 00:02:05,315] [INFO] [real_accelerator.py:158:get_accelerator] Setting ds_accelerator to cuda (auto detect)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

SELECT p.name, SUM(CASE WHEN EXTRACT(YEAR FROM s.sale_date) = 2022 THEN 1 ELSE 0 END) AS sales_2022, SUM(CASE WHEN EXTRACT(YEAR FROM s.sale_date) = 2021 THEN 1 ELSE 0 END) AS sales_2021, (SUM(CASE WHEN EXTRACT(YEAR FROM s.sale_date) = 2022 THEN 1 ELSE 0 END) - SUM(CASE WHEN EXTRACT(YEAR FROM s.sale_date) = 2021 THEN 1 ELSE 0 END)) AS change_in_sales FROM products p JOIN sales s ON p.product_id = s.product_id GROUP BY p.name HAVING (SUM(CASE WHEN EXTRACT(YEAR FROM s.sale_date) = 2022 THEN 1 ELSE 0 END) - SUM(CASE WHEN EXTRACT(YEAR FROM s.sale_date) = 2021 THEN 1 ELSE 0 END)) > 0 ORDER BY (SUM(CASE WHEN EXTRACT(YEAR FROM s.sale_date) = 2022 THEN 1 ELSE 0 END) - SUM(CASE WHEN EXTRACT(YEAR FROM s.sale_date) = 2021 THEN 1 ELSE 0 END)) DESC LIMIT 1;


In [2]:
# !pip install torch transformers bitsandbytes accelerate

In [3]:
%%time 
inputs = tokenizer(prompt, return_tensors="pt").to('cuda')
generated_ids = model.generate(
    **inputs,
    num_return_sequences=1,
    eos_token_id=eos_token_id,
    pad_token_id=eos_token_id,
    max_new_tokens=100,
    do_sample=False,
    num_beams=5
)

outputs = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)
torch.cuda.empty_cache()
print(outputs[0].split("```sql")[-1].split("```")[0].split(";")[0].strip() + ";")

SELECT p.name, SUM(CASE WHEN EXTRACT(YEAR FROM s.sale_date) = 2022 THEN 1 ELSE 0 END) AS sales_2022, SUM(CASE WHEN EXTRACT(YEAR FROM s.sale_date) = 2021 THEN 1 ELSE 0 END) AS sales_2021, (SUM(CASE WHEN;
CPU times: user 42.9 s, sys: 82.7 ms, total: 42.9 s
Wall time: 43 s


In [2]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

[2023-11-08 20:57:39,093] [INFO] [real_accelerator.py:158:get_accelerator] Setting ds_accelerator to cuda (auto detect)


In [3]:
torch.cuda.is_available()

True

In [4]:
# use an A100 on Colab Pro (or any system with >30GB VRAM on your own machine) to load this in bf16
# if not available, use a GPU with minimum 20GB VRAM to load this in 8bit, or with minimum 12GB of VRAM to load in 4bit
# on Colab, works with a V100 but crashes on a T4

# downloading the model and then loading it to memory step takes around 10 minutes the first time. Be patient :)

In [5]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("defog/sqlcoder-7b")
model = torch.nn.DataParallel(AutoModelForCausalLM.from_pretrained("defog/sqlcoder-7b", load_in_4bit=True,device_map="auto",)).to('cuda')

RuntimeError: Failed to import transformers.integrations.bitsandbytes because of the following error (look up to see its traceback):
[Errno 13] Permission denied: '/root/google_vm_config.lock'

In [ ]:
question = "What products has the biggest fall in sales in 2022 compared to 2021? Give me the product name, the sales amount in both years, and the difference."

In [ ]:
prompt = """### Instructions:
Your task is convert a question into a SQL query, given a Postgres database schema.
Adhere to these rules:
- **Deliberately go through the question and database schema word by word** to appropriately answer the question
- **Use Table Aliases** to prevent ambiguity. For example, `SELECT table1.col1, table2.col1 FROM table1 JOIN table2 ON table1.id = table2.id`.
- When creating a ratio, always cast the numerator as float

### Input:
Generate a SQL query that answers the questioq `{question}`.
This query will run on a database whose schema is represented in this string:
CREATE TABLE products (
  product_id INTEGER PRIMARY KEY, -- Unique ID for each product
  name VARCHAR(50), -- Name of the product
  price DECIMAL(10,2), -- Price of each unit of the product
  quantity INTEGER  -- Current quantity in stock
);

CREATE TABLE customers (
   customer_id INTEGER PRIMARY KEY, -- Unique ID for each customer
   name VARCHAR(50), -- Name of the customer
   address VARCHAR(100) -- Mailing address of the customer
);

CREATE TABLE salespeople (
  salesperson_id INTEGER PRIMARY KEY, -- Unique ID for each salesperson
  name VARCHAR(50), -- Name of the salesperson
  region VARCHAR(50) -- Geographic sales region
);

CREATE TABLE sales (
  sale_id INTEGER PRIMARY KEY, -- Unique ID for each sale
  product_id INTEGER, -- ID of product sold
  customer_id INTEGER,  -- ID of customer who made purchase
  salesperson_id INTEGER, -- ID of salesperson who made the sale
  sale_date DATE, -- Date the sale occurred
  quantity INTEGER -- Quantity of product sold
);

CREATE TABLE product_suppliers (
  supplier_id INTEGER PRIMARY KEY, -- Unique ID for each supplier
  product_id INTEGER, -- Product ID supplied
  supply_price DECIMAL(10,2) -- Unit price charged by supplier
);

-- sales.product_id can be joined with products.product_id
-- sales.customer_id can be joined with customers.customer_id
-- sales.salesperson_id can be joined with salespeople.salesperson_id
-- product_suppliers.product_id can be joined with products.product_id

### Response:
Based on your instructions, here is the SQL query I have generated to answer the question `{question}`:
```sql
""".format(question=question)

In [ ]:
eos_token_id = tokenizer.convert_tokens_to_ids(["```"])[0]

In [ ]:
# excruciatingly slow on an V100 with 4bit quantization
# takes around 1-2 minutes per query
# on a single A100 40GB, takes ~10-20 seconds

In [ ]:
import torch
import torch.nn as nn

In [ ]:
inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
generated_ids = model.module.generate(
    **inputs,
    num_return_sequences=1,
    eos_token_id=eos_token_id,
    pad_token_id=eos_token_id,
    max_new_tokens=400,
    do_sample=False,
    num_beams=5
)

In [ ]:
outputs = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)

In [ ]:
torch.cuda.empty_cache()
# empty cache so that you do generate more results w/o memory crashing
# particularly important on Colab – memory management is much more straightforward
# when running on an inference service

In [ ]:
print(outputs[0].split("```sql")[-1].split("```")[0].split(";")[0].strip() + ";")